# SVD decomposition with `scipy`

Import the required modules

In [ ]:
import scipy.linalg as la
import numpy as np

Generate a random 5x4 matrix

In [ ]:
np.random.seed(0)

A = np.random.rand(5,4)
A

## `<numpy|scipy>.linalg.svd` 

Equivalent implementations:
- https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html



### Full SVD (`full_matrices = True` - default)

**input:**

$A \in \mathbb{R}^{m\times n}$

**output:**

$U \in \mathbb{R}^{m\times m}, S \in \mathbb{R}^{m\times n}, V \in \mathbb{R}^{n\times n}$

or, more precisely

$\boldsymbol{\sigma} \in \mathbb{R}^{q} $

where $q = \min(m,n)$.


In [ ]:
U, s, VT = np.linalg.svd(A)
#U, s, VT = la.svd(A)
print('U shape: ', U.shape)
print('s shape: ', s.shape)
print('VT shape: ', VT.shape)

Build the matrix $S$

In [ ]:
S = np.zeros(A.shape)
for i in range(len(s)):
    S[i, i] = s[i]
S

In [ ]:
S = la.diagsvd(s, A.shape[0], A.shape[1])
S

Reconstruct the matrix $A$

In [ ]:
A_svd = np.matmul(U, np.matmul(S,VT))
# equivalently: A_svd = U @ S @ VT
print(f"err: {(la.norm(A - A_svd) / la.norm(A))}")

### Thin SVD (`full_matrices = False`)

**input:**

$A \in \mathbb{R}^{m\times n}$

**output:**

$U \in \mathbb{R}^{m\times q}, S \in \mathbb{R}^{q\times q}, V \in \mathbb{R}^{n\times q}$

or, more precisely

$\boldsymbol{\sigma} \in \mathbb{R}^{q} $

where $q = \min(m,n)$.



In [ ]:
U, s, VT = la.svd(A, full_matrices=False)
print('U shape: ', U.shape)
print('s shape: ', s.shape)
print('VT shape: ', VT.shape)

Build the matrix $S$

In [ ]:
S = np.diag(s)
S

Reconstruct the matrix $A$

In [ ]:
A_svd = np.matmul(U, np.matmul(S,VT))
print(f"err: {la.norm(A - A_svd) / la.norm(A)}")

### A note on vectorization
Vectorization refers to the practice of replacing explicit loops with high-level mathematical operations that act on entire arrays or matrices at once. This leads to much better performance because it replaces slow Python loops with fast, optimized C and Fortran operations.

Indeed, we could be inclined to reconstruct $A_k$ with a for loop and the explicit formula
$$A_k = \sigma_1 u_1 v_1^T + ... + \sigma_k u_k v_k^T.$$

Let's measure the time taken for this operation for a matrix $A$ that is a bit larger

In [ ]:
import time

A = np.random.rand(1000, 1500)
U, s, VT = la.svd(A, full_matrices=False)
S = np.diag(s)


Time the reconstruction with a for loop

In [ ]:
start_time = time.time()

A_reconstructed_loop = np.zeros_like(A)
for i in range(len(s)):
    A_reconstructed_loop += s[i] * np.outer(U[:, i], VT[i, :])

loop_time = time.time() - start_time

Time the vectorized reconstruction using matrix multiplication

In [ ]:
start_time = time.time()

A_reconstructed_matmult = U @ S @ VT

matmult_time = time.time() - start_time

If $S \in \mathbb R^{q \times q}$ diagonal (with elements on the diagonal equal to $s \in \mathbb R^{q}$), then the each of $US$ is equal to the element wise product of that row of $U$ with $s$, namely, $(US)_{i,j} = U_{i, j} s_j$. Indeed, $(US)_{i,j} = \sum_{k} U_{i, k} S_{k, j} = U_{i, j} S_{j, j} = U_{i, j} s_j$, since $S_{k, j} = 0$ iff $k \neq j$.

In [ ]:
start_time = time.time()

# here we are using broadcasting to avoid the creation of a diagonal matrix
# see: https://numpy.org/doc/stable/user/basics.broadcasting.html
A_reconstructed_vectorized = (U * s) @ VT

vectorized_time = time.time() - start_time

We compare the results

In [ ]:
print(f"Time for reconstruction using for loop: {loop_time:.6f} seconds")
print(f"Time for vectorized reconstruction: {vectorized_time:.6f} seconds")
print(f"Matmult is {loop_time / matmult_time:.1f} times faster than the loop")
print(f"Vectorized is {matmult_time / vectorized_time:.1f} times faster than the matmult")

difference = np.abs(A_reconstructed_loop - A_reconstructed_vectorized).max()
print(f"Difference between the two reconstructions: {difference:.6e}")